# 사전 학습된 FastText를 BlazingText에 호스팅하기
*본 노트북 예제는 [사전 학습된 FastText를 BlazingText에 호스팅하기(영어)](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_amazon_algorithms/blazingtext_hosting_pretrained_fasttext/blazingtext_hosting_pretrained_fasttext.ipynb) 노트북을 번역하면서 한국어 컨텐츠를 추가하였습니다.*

## 개요

이 노트북에서는 BlazingText가 [FastText 모델](https://fasttext.cc/docs/en/english-vectors.html)로
사전 학습된(pre-trained) 텍스트 분류(Text Classification) 및 Word2Vec 모델 (FastText 모델)의 호스팅을 지원하는 방법을 보여줍니다.

BlazingText는 supervised 모드 사용 시 GPU 가속 버전의 FastText입니다. FastText는 워드 임베딩(unsupervisied)과 텍스트 분류(supervised)를 모두 수행하는 데 사용되는 얕은 신경망 모델(shallow Neural Network model)입니다. BlazingText는 사용자 정의 CUDA 커널을 사용하여 FastText의 학습 과정을 가속화하지만 기본 알고리즘은 두 알고리즘에서 동일합니다.

따라서 FastText로 학습된 모델이 있거나 FastText 팀에서 제공하는 사전 학습된 모델 중 하나가 사용 사례에 충분하면 BlazingText에 대한 호스팅 기능을 활용하여 실시간 예측을 위한 SageMaker 엔드포인트를 설정할 수 있습니다. 다시 말해, FastText로 사전 학습된 모델을 이미 보유하고 있다면 BlazingText 알고리즘으로 다시 학습하지 않아도 됩니다.

In [1]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import json

sess = sagemaker.Session()

role = get_execution_role()
#print(role) # This is the role that SageMaker would use to leverage AWS resources (S3, CloudWatch) on your behalf

#bucket = sess.default_bucket() # Replace with your own bucket name if needed
bucket = 'blazingtext-hol-daekeun'
prefix = 'sagemaker/DEMO-blazingtext-pretrained-fasttext' #Replace with the prefix under which you want to store the data if needed

In [2]:
region_name = boto3.Session().region_name
container = sagemaker.amazon.amazon_estimator.get_image_uri(region_name, "blazingtext", "latest")
print('Using SageMaker BlazingText container: {} ({})'.format(container, region_name))

Using SageMaker BlazingText container: 811284229777.dkr.ecr.us-east-1.amazonaws.com/blazingtext:latest (us-east-1)


### Hosting the [Language Idenfication model](https://fasttext.cc/docs/en/language-identification.html) by FastText

이 노트북에서는 언어 식별(Language Identification)을 위해 FastText가 제공하는 사전 학습된 모델을 활용합니다. 언어 식별은 입력 텍스트의 언어 식별 후 다양한 다운스트림 작업들에 해당 언어의 특정 모델을 적용해야 하는 많은 자연어 처리(NLP; Natural Language Processing) 응용 프로그램의 첫 단계입니다. 이 언어 식별 모델은 클래스 레이블(class label)로 언어 ID(language ID)를 사용하는 텍스트 분류 모델이므로 FastText를 교육에 직접 사용할 수 있습니다. FastText로 사전 학습된 언어 모델은 176개의 다른 언어 식별을 지원하며 한국어도 당연히 포함되어 있습니다.

사전 학습된 모델 [1] 을 로컬 인스턴스로 가져오기 위해, [FastText 웹사이트](https://fasttext.cc/docs/en/language-identification.html)의 언어 식별 모델을 다운로드합니다.
FastText 관련 논문들은 아래를 참조해 주세요.

[1] A. Joulin, E. Grave, P. Bojanowski, T. Mikolov, Bag of Tricks for Efficient Text Classification<br>
[2] P. Bojanowski, E. Grave, A. Joulin, T. Mikolov, Enriching Word Vectors with Subword Information<br>
[3] A. Joulin, E. Grave, P. Bojanowski, M. Douze, H. Jégou, T. Mikolov, FastText.zip: Compressing text classification models

In [3]:
!wget -O model.bin https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin

--2019-11-06 06:58:05--  https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.6.166, 104.20.22.166, 2606:4700:10::6814:16a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.6.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131266198 (125M) [application/octet-stream]
Saving to: ‘model.bin’

model.bin           100%[===================>] 125.18M  23.0MB/s    in 6.0s    

2019-11-06 06:58:12 (20.7 MB/s) - ‘model.bin’ saved [131266198/131266198]



## Local Inference

엔드포인트를 생성하기 전에 fasttext 라이브러리를 설치하여 로컬에서 추론을 수행해 보겠습니다.

In [4]:
!pip install fasttext

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
import fasttext
model = fasttext.load_model('model.bin')

In [6]:
print(model.words[0:100]) # list of words in dictionary
print(model['안녕하세요']) # get the vector of the word 'king'

['</s>', 'the', 'de', 'a', 'in', 'of', 'and', 'to', 'la', 'sa', 'en', 'is', 'i', 'в', 'der', '-', 'was', 'und', 'и', 'die', 'for', 'on', 'на', 'The', 'as', 'que', 'e', 'di', 'un', 'el', 'that', 'by', 'des', 'with', 'le', 'nga', 'del', 'an', 'y', 'et', '#', 'den', 'at', 'se', '–', ':', '(UTC)', 'von', 'à', 'from', 'o', 'I', 'it', 'da', 'il', '—', 'Ang', 'do', 'du', 'be', 'his', 'van', 'les', 'na', 'are', 'una', 'he', 'не', ',', 'er', 'w', 'al', 'In', 'es', 'är', 'das', 'im', 'per', 'this', 'con', 'mit', 'no', 'ist', 'not', 'Tom', 'с', 'km', 'or', 'je', 'La', 'por', '.', 'est', 'A', 'als', 'ang', 'zu', 'los', 'have', 'has']
[ 0.30915835  0.2186219  -0.06362805  0.2009838  -0.06676163  1.2262515
 -0.03246042  0.25003538 -0.16305555  0.04806601 -0.9093998   0.1976639
  0.30237058  0.66566044 -0.06046864  0.1827585 ]


간단한 추론을 수행해 봅니다. 테스트할 문장은 한국어, 영어, 일본어, 독일어, 스페인어입니다.

In [7]:
sentences = [
    "머신 러닝을 배우고 싶어요.",
    "I want to learn machine learning.",
    "マシンラーニングを学びたいです。",
    "Ich möchte maschinelles Lernen lernen.",
    "Quiero aprender el aprendizaje automático."]

In [8]:
model.predict(sentences)

([['__label__ko'],
  ['__label__en'],
  ['__label__ja'],
  ['__label__de'],
  ['__label__es']],
 array([[1.00001645],
        [0.9821412 ],
        [1.00004888],
        [0.99612123],
        [0.97240931]]))

모든 언어들이 올바르게 식별된 것을 확인할 수 있습니다.

## Creating SageMaker Inference Endpoint

BlazingText 컨테이너를 사용하여 SageMaker 추론 엔드포인트를 생성합니다. 이 엔드포인트는 FastText에서 제공하는 사전 학습된 모델과 호환되며 아무 수정 없이 직접 추론에 사용할 수 있습니다. 추론 엔드포인트는 컨텐츠 유형의 `application/json`에서 작동합니다.

`model.bin` 파일을 `langid.tar.gz`로 압축하고 S3에 업로드합니다. 로컬 인스턴스에 저장된 모델 파일은 더 이상 필요하지 않으므로 삭제합니다.

In [9]:
!tar -czvf langid.tar.gz model.bin
model_location = sess.upload_data("langid.tar.gz", bucket=bucket, key_prefix=prefix)
!rm langid.tar.gz model.bin

model.bin


엔드포인트를 생성합니다. 약 5-10분 정도 소요됩니다.

In [11]:
lang_id = sagemaker.Model(model_data=model_location, image=container, role=role, sagemaker_session=sess)
lang_id.deploy(initial_instance_count = 1,instance_type = 'ml.m4.xlarge')
predictor = sagemaker.RealTimePredictor(endpoint=lang_id.endpoint_name, 
                                   sagemaker_session=sess,
                                   serializer=json.dumps,
                                   deserializer=sagemaker.predictor.json_deserializer)

--------------------------------------------------------------------------------------------------!

이제 엔드포인트에서 직접 추론을 수행해 보겠습니다.

In [12]:
payload = {"instances" : sentences}

In [13]:
predictions = predictor.predict(payload)
print(predictions)

[{'prob': [1.000016450881958], 'label': ['__label__ko']}, {'prob': [0.9821411967277527], 'label': ['__label__en']}, {'prob': [1.0000488758087158], 'label': ['__label__ja']}, {'prob': [0.9961212277412415], 'label': ['__label__de']}, {'prob': [0.9724093079566956], 'label': ['__label__es']}]


FastText는 클래스 레이블 앞에 `__label__`이 붙기 때문에, FastText에서 제공하는 사전 학습된 모델로 추론을 수행할 때 출력 레이블 앞에 `__label__`이 붙는 것을 알 수 있습니다. <br>
약간의 전처리를 통해 `__label__` 접두사(prefix)를 제거할 수 있습니다.

In [14]:
import copy
predictions_copy = copy.deepcopy(predictions) # Copying predictions object because we want to change the labels in-place
for output in predictions_copy:
    output['label'] = output['label'][0][9:].upper() #__label__ has length of 9

print(predictions_copy)

[{'prob': [1.000016450881958], 'label': 'KO'}, {'prob': [0.9821411967277527], 'label': 'EN'}, {'prob': [1.0000488758087158], 'label': 'JA'}, {'prob': [0.9961212277412415], 'label': 'DE'}, {'prob': [0.9724093079566956], 'label': 'ES'}]


### Stop / Close the Endpoint (Optional)
실시간 예측을 제공하기 위해 엔드포인트를 계속 실행할 필요가 없는 경우, 불필요한 과금을 막기 위해 엔드포인트를 삭제해야 합니다.

In [15]:
sess.delete_endpoint(predictor.endpoint)

이와 유사하게, 여러분은 사전 학습된 [FastText word2vec 모델](https://fasttext.cc/docs/en/pretrained-vectors.html) SageMaker BlazingText hosting을 이용해 호스팅할 수 있습니다.